In [ ]:
import sys
sys.path.append('/media/senilab/DATA/Master/I3D-Tensorflow/')
import os
import time
import numpy
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import input_test
import math
import numpy as np
# from i3dCopy1 import InceptionI3d
from i3d import InceptionI3d
from utils import *
import pandas as pd
import cv2
from PIL import Image
import hickle
from os import listdir 
from os.path import isfile, join, isdir

In [ ]:
def multihead_attention(queries, 
                        keys, 
                        num_units=None, 
                        num_heads=64, 
                        dropout_rate=0,
                        is_training=True,
                        causality=False,
                        scope="multihead_attention", 
                        reuse=None):
    

    '''Applies multihead attention.
    
    Args:
      queries: A 3d tensor with shape of [N, T_q, C_q].
      keys: A 3d tensor with shape of [N, T_k, C_k].
      num_units: A scalar. Attention size.
      dropout_rate: A floating point number.
      is_training: Boolean. Controller of mechanism for dropout.
      causality: Boolean. If true, units that reference the future are masked. 
      num_heads: An int. Number of heads.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
        
    Returns
      A 3d tensor with shape of (N, T_q, C)  
    '''
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
        # Set the fall back option for num_units
        if num_units is None:
            num_units = queries.get_shape().as_list[-1]
        
        # Linear projections
        Q = tf.contrib.layers.fully_connected(queries, num_units) # (N, T_q, C)
        K = tf.contrib.layers.fully_connected(queries, num_units ) # (N, T_k, C)
        V = tf.contrib.layers.fully_connected(keys, num_units) # (N, T_k, C)
        
        Q1 = tf.reshape(Q,(Q.get_shape().as_list()[0],Q.get_shape().as_list()[1],num_units))
        K1 = tf.reshape(K,(Q.get_shape().as_list()[0],Q.get_shape().as_list()[1],num_units))        
        V1 = tf.reshape(V,(Q.get_shape().as_list()[0],Q.get_shape().as_list()[1],num_units))
        
        
        # Split abboxfnd concat
        Q_ = tf.concat(tf.split(Q1, num_heads, axis = 2),axis =0) # (h*N, T_q, C/h) 
        K_ = tf.concat(tf.split(K1, num_heads, axis = 2),axis =0) # (h*N, T_k, C/h) 
        V_ = tf.concat(tf.split(V1, num_heads, axis = 2),axis =0) # (h*N, T_k, C/h) 

        # Multiplication
        outputs = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1])) # (h*N, T_q, T_k)
        
        # Scale
        outputs = outputs / (K_.get_shape().as_list()[-1] ** 0.5)

                
        # Key Masking
        key_masks = tf.sign(tf.abs(tf.reduce_sum(keys, axis=-1))) # (N, T_k)
        key_masks = tf.tile(key_masks, [num_heads, 1]) # (h*N, T_k)
        key_masks = tf.tile(tf.expand_dims(key_masks, 1), [1, tf.shape(queries)[1], 1]) # (h*N, T_q, T_k)
       # 
        paddings = tf.ones_like(outputs)*(-2**32+1)
        outputs = tf.where(tf.equal(key_masks, 0), paddings, outputs) # (h*N, T_q, T_k)
          
        # Causality = Future blinding
        if causality:
            diag_vals = tf.ones_like(outputs[0, :, :]) # (T_q, T_k)
            tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense() # (T_q, T_k)
            masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(outputs)[0], 1, 1]) # (h*N, T_q, T_k)
  # 
            paddings = tf.ones_like(masks)*(-2**32+1)
            outputs = tf.where(tf.equal(masks, 0), paddings, outputs) # (h*N, T_q, T_k)
  
        # Activation
        outputs = tf.nn.softmax(outputs) # (h*N, T_q, T_k)
        matt    = outputs
        
        
        # Query Masking
        query_masks = tf.sign(tf.abs(tf.reduce_sum(queries, axis=-1))) # (N, T_q)
        query_masks = tf.tile(query_masks, [num_heads, 1]) # (h*N, T_q)
        query_masks = tf.tile(tf.expand_dims(query_masks, -1), [1, 1, tf.shape(keys)[1]]) # (h*N, T_q, T_k)
        outputs *= query_masks # broadcasting. (N, T_q, C)
          
        # Dropouts
        outputs = tf.contrib.layers.dropout(outputs, keep_prob=dropout_rate, is_training=tf.convert_to_tensor(is_training))
               
        # Weighted sum
        outputs = tf.matmul(outputs, V_) # ( h*N, T_q, C/h)
        
        # Restore shape
        outputs = tf.concat(tf.split(outputs, num_heads,axis = 0),axis =2 ) # (N, T_q, C)
              
        # Residual connection
        outputs += queries              
              
        # Normalize
        outputs = normalize(outputs) # (N, T_q, C)
 
    return outputs, matt

In [ ]:
def normalize(inputs, 
              epsilon = 1e-8,
              scope="ln",
              reuse=None):
    '''Applies layer normalization.
    
    Args:
      inputs: A tensor with 2 or more dimensions, where the first dimension has
        `batch_size`.
      epsilon: A floating number. A very small number for preventing ZeroDivision Error.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
      
    Returns:
      A tensor with the same shape and data dtype as `inputs`.
    '''
    with tf.variable_scope(scope, reuse=reuse):
        inputs_shape = inputs.get_shape()
        params_shape = inputs_shape[-1:]
    
        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
        beta= tf.Variable(tf.zeros(params_shape))
        gamma = tf.Variable(tf.ones(params_shape))
        normalized = (inputs - mean) / ( (variance + epsilon) ** (.5) )
        outputs = gamma * normalized + beta
        
    return outputs

In [ ]:
def positional_encoding(inputs,
                        maxlen,
                        masking=True,
                        scope="positional_encoding"):
    '''Sinusoidal Positional_Encoding. See 3.5
    inputs: 3d tensor. (N, T, E)
    maxlen: scalar. Must be >= T
    masking: Boolean. If True, padding positions are set to zeros.
    scope: Optional scope for `variable_scope`.
    returns
    3d tensor that has the same shape as inputs.
    '''

    E = inputs.get_shape().as_list()[-1] # static
    N, T = tf.shape(inputs)[0], tf.shape(inputs)[1] # dynamic
    with tf.variable_scope(scope, reuse=True):
        # position indices
        position_ind = tf.tile(tf.expand_dims(tf.range(T), 0), [N, 1]) # (N, T)

        # First part of the PE function: sin and cos argument
        position_enc = np.array([
            [pos / np.power(10000, (i-i%2)/E) for i in range(E)]
            for pos in range(maxlen)])

        # Second part, apply the cosine to even columns and sin to odds.
        position_enc[:, 0::2] = np.sin(position_enc[:, 0::2])  # dim 2i
        position_enc[:, 1::2] = np.cos(position_enc[:, 1::2])  # dim 2i+1
        position_enc = tf.convert_to_tensor(position_enc, tf.float32) # (maxlen, E)

        # lookup
        outputs = tf.nn.embedding_lookup(position_enc, position_ind)

        # masks
        if masking:
            outputs = tf.where(tf.equal(inputs, 0), inputs, outputs)

        return tf.to_float(outputs)

In [ ]:
datasavetry = '/media/senilab/DATA2/array_hmdb/train_rgb/Veoh_Alpha_Dog_1_walk_f_cm_np1_ba_med_7'
add = hickle.load(datasavetry)

In [ ]:
train_dir_rgb = '/media/senilab/DATA2/array_hmdb/train_rgb/'
train_dir_flow = '/media/senilab/DATA2/array_hmdb/train_flow/'

In [ ]:
list_train_rgb = sorted([f for f in listdir(train_dir_rgb) if isfile(join(train_dir_rgb, f))])
list_train_rgb

In [ ]:
for i in list_train_rgb:
    data_file = train_dir_rgb + i
    data = hickle.load(data_file)
    n_frames = 64
    n_hidden = 512

    group1 = data['feat']
    position_idx  = tf.ones((n_frames,1*n_hidden))
    positional_encode_tmp = positional_encoding(tf.expand_dims(position_idx, axis=0), n_frames) 
    group1 = group1 + positional_encode_tmp

    multi_g1, matt1  = multihead_attention(queries=group1, keys=group1, num_units=n_hidden, num_heads=64,dropout_rate=0.9,is_training=1,causality=True, scope = 'forward', reuse = tf.AUTO_REUSE)
    
    

In [ ]:
n_frames = 64
n_hidden = 512

label = data['label']
group1 = data['feat']
position_idx  = tf.ones((n_frames,1*n_hidden))
positional_encode_tmp = positional_encoding(tf.expand_dims(position_idx, axis=0), n_frames) 
group1 = group1 + positional_encode_tmp

multi_g1, matt1  = multihead_attention(queries=group1, keys=group1, num_units=n_hidden, num_heads=64,dropout_rate=0.9,is_training=1,causality=True, scope = 'forward', reuse = tf.AUTO_REUSE)

In [ ]:
matt2 = tf.concat([multi_g1, multi_g1], 0) 
matt2

In [ ]:
# WandB – Config is a variable that holds and saves hyperparameters and inputs
    # Initialize config
batch_size = 4         # Input batch size for training
learning_rate = 0.0001    # Learning rate
num_steps = 25000       # Number of batches to train
display_step = 100      # Steps between displaying output
num_input = 64         #(img shape: 64 * 64 * 64 = 262144)
num_classes = 51        #(0-9 categories)
n_hidden_1 = 512
n_hidden_2 = 64

In [ ]:
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [ ]:
learning_rate = 0.5
epochs = 10
batch_size = 100

# declare the training data placeholders
# input x - for 28 x 28 pixels = 784
x = tf.placeholder(tf.float32, [None, 64])
# now declare the output data placeholder - 10 digits
y = tf.placeholder(tf.float32, [None, 51])


# now declare the weights connecting the input to the hidden layer
W1 = tf.Variable(tf.random_normal([64, 512], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([300]), name='b1')
# and the weights connecting the hidden layer to the output layer
W2 = tf.Variable(tf.random_normal([300, 10], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([10]), name='b2')

hidden_out = tf.add(tf.matmul(multi_g1, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

In [ ]:
# Create model
def neural_net(x):
    
    # Hidden fully connected layer with 128 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
#     layer_1 = tf.add(tf.tensordot(tf.cast(x, tf.float32), weights['h1'], [[2], [0]]), biases['b1'])
    
    # Hidden fully connected layer with 128 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    
    return out_layer

In [ ]:
hasil = neural_net(multi_g1)